In [ ]:
!nvidia-smi

NotImplementedError: ignored

Install the libraries

In [ ]:
!pip install -q langchain transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.1 MB/s eta 0:00:00


Load the libraries

In [ ]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap


Download the Model - We are using NousResearch's Llama2 which is the same as Meta AI's Llama 2, the only difference being "**Not requiring authentication to download**"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Define Transformers Pipeline which will be fed into Langchain

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 1024,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )


Define the Prompt format for Llama 2 - This might change if you have a different model

In [ ]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are an expert Technical Recruiter(named Clark) who has hired the best candidates for the top companies.
Your job is to assess the candidates by asking them various technical and non technical questions to check how fit they are for a particular role/job. Youshould not be harmful, unethical, racist, sexist, toxic or dangerous. Please ensure that your Questions are able to test the candidates thoroughly.
Do not create false questions or ask them. Be unbiased, do not help the  candidates unless they have answered the question.As soon as the candidate mentions his/her interested role, you greet them and start asking questions.  """


All the helper fucntions to generate prompt, prompt template, clean up output text

In [ ]:

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

Defining Langchain LLM

In [ ]:

llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.1,'max_length': 1024, 'top_k' :50})



In [ ]:
system_prompt = "generate multiple choice objective type questions to test the ability of candidates. The questions should be of Extreme difficuilty. Dive deeper into technical aspects bsed on the role said by candidate.The format should be: Question:\nOptions:\nAnswer:\n, generate atleast 10 questions"
# system_prompt = '''generate multiple choice objective type questions to test the ability of candidates. The questions should be of Extreme difficuilty. Dive deeper into technical aspects bsed on the role said by candidate.The format should be a json: {Question:{question},Option_A:{Option A},Question_B:{question B} ,Option_C:{Option C},Answer:{Correct Option}} generate atleast 5 questions'''
instruction = "Use the following text from the user, to identify the job role they applied for and then test the concepts as above:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)


[INST]<>
generate multiple choice objective type questions to test the ability of candidates. The questions should be of Extreme difficuilty. Dive deeper into technical aspects bsed on the role said by candidate.The format should be: Question:
Options:
Answer:
, generate atleast 10 questions
<>

Use the following text from the user, to identify the job role they applied for and then test the concepts as above:

 {text}[/INST]


In [ ]:
prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = False)



In [ ]:
text = "Hi, I'd like to apply for the senior machine learning enineerg role"

In [ ]:
response = llm_chain.run(text)
print(response)

  Great, based on your job role application, I will generate 10 multiple choice objective type questions to test your ability in the field of senior machine learning engineering. Here are the questions:

Question 1:
Determine the appropriate algorithm for solving a classification problem, given:

Options:
A) Decision Trees
B) Random Forests
C) Support Vector Machines (SVM)
D) Neural Networks

What is your answer?

Question 2:
Design an experiment to evaluate the performance of a machine learning model on a new dataset. The dataset contains 1000 samples and 10 features. The model is trained on a dataset with 800 samples and 8 features. How would you:

Options:
A) Use cross-validation to evaluate the model's performance
B) Use a hold-out approach to evaluate the model's performance
C) Use a combination of both cross-validation and hold-out to evaluate the model's performance
D) Use a different approach that you think is suitable for this scenario

What is your answer?

Question 3:
Given 